# Stability analysis - full elastic net on 150 WGS&RNAseq features

In [57]:
rm(list=ls())

In [58]:
#setwd("/Volumes/igmm/HGS-OvarianCancerA-SGP-WGS/rnaseq/analysis/SHGSOC")
library(tximport)
require(DESeq2)
library(caret)
library(caTools)

In [59]:
load("~/Desktop/BRCA1_BRCA2_SVs_paper/Analysis/Robjecttosetup_multivariablemodel_full.RData")


In [60]:

df<-numeric()
perf_list<-list(50)

performance_metrics = function(actual, predicted) {
  acc<-sum(actual == predicted)/length(actual)
  TP<-  sum(actual=='HRD'& predicted=='HRD')
  TN<-  sum(actual=='HRP'& predicted=='HRP')
  FP<-  sum(actual=='HRD'& predicted=='HRP')
  FN<-  sum(actual=='HRP'& predicted=='HRD')  
  sensitivity<-TP/(TP+FN)  
  specificity<-TN/(TN+FP)
  auc<-  colAUC(as.numeric(predict(model, test.data2)), test.data2$HRDeficient)
  r<-list(Accuracy=acc,Sensitivity=sensitivity,Specificity=specificity,AUC=auc[1])
    return(r)
}

i<-0
while (length(df) <1500 | length(df)==0 ){
  i<-i+1

    
print(i)
print(": Setting seed...")
set.seed(489+i)

print("Partitioning data...")
train = sample(1:nrow(sampleInfo_analysis), nrow(sampleInfo_analysis)*0.8)
train.data<-sampleInfo_analysis[train,]
test.data<-sampleInfo_analysis[-train,]

if (dim(table(train.data$BRCA1_LOH))==1){
    next
}
    
print("Identify DE genes...")
HRD<-intersect(which(coldata$condition!="Excluded"),which(coldata$Sample %in% rownames(train.data)))
ddsHRD<-dds[,HRD]
colData(ddsHRD)$condition<-droplevels(colData(ddsHRD)$condition)
ddsHRD <- DESeq(ddsHRD)
res<-results(ddsHRD)
    
biom_res<-read.table("~/Desktop/All_genes_quant_type.txt",sep="\t",header=T)
rownames(biom_res)<-as.character(biom_res[,1])
protein_coding_genes<-as.character(biom_res[biom_res$Gene.type=="protein_coding",1])
    
resLFC <- lfcShrink(ddsHRD[protein_coding_genes,], coef="condition_HRProf_vs_HRDef",type="ashr")
resLFC$new.padj<-p.adjust(resLFC$pvalue, method="BH",n=length(protein_coding_genes))
res001<- resLFC[is.na(resLFC$padj)==FALSE & resLFC$new.padj<0.05 & abs(resLFC$log2FoldChange)>1,]
res001_ord<-res001[order(res001$new.padj),]
DE_genes_ensembl<-biom_res[rownames(res001_ord),1]


print("Calculating expression sig...")
DEhrd_vsd<-assay(vsd)[as.character(DE_genes_ensembl),]

HRDcomp_train<-which(coldata$Sample %in% rownames(train.data) )
p.train<-prcomp(t(DEhrd_vsd[,HRDcomp_train]))

HRDcomp_test<-which(coldata$Sample %in% rownames(test.data) )
p.test<-prcomp(t(DEhrd_vsd[,HRDcomp_test]))

de_df_tr<-data.frame(Sample=coldata[HRDcomp_train,"Sample"],HRD_DE_sig=p.train$x[,"PC1"])
de_df_test<-data.frame(Sample=coldata[HRDcomp_test,"Sample"],HRD_DE_sig=p.test$x[,"PC1"])

train.data$Sample<-rownames(train.data)
train.data2<-merge(train.data,de_df_tr,by="Sample")
train.data2<-train.data2[,setdiff(colnames(train.data2),"Sample")]

test.data$Sample<-rownames(test.data)
test.data2<-merge(test.data,de_df_test,by="Sample")
test.data2<-test.data2[,setdiff(colnames(test.data2),"Sample")]

train.data2$HRD_DE_sig<- -train.data2$HRD_DE_sig
test.data2$HRD_DE_sig<- -test.data2$HRD_DE_sig   

print("Fit elastic net...")
model <- train(
  HRDeficient ~., data = train.data2, method = "glmnet",
  trControl = trainControl("cv", number = 10,
                           summaryFunction = twoClassSummary,
                           classProbs = TRUE),
  standardize=TRUE,                          
  tuneLength = 10
)

print("Get coefficients...")
res<-coef(model$finalModel, model$bestTune$lambda)

df<-cbind(df,as.numeric(res))
    
perf_list[i]<-performance_metrics(actual = test.data2$HRDeficient,
         predicted = predict(model, newdata = test.data2))
    
}
save(df,file="~/Desktop/df_1_50_loh_full.RData")
save(perf_list,file="~/Desktop/df_1_50_perf_loh_full.RData")

[1] 1
[1] ": Setting seed..."
[1] "Partitioning data..."
[1] "Identify DE genes..."


estimating size factors
using 'avgTxLength' from assays(dds), correcting for library size
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041


[1] "Calculating expression sig..."
[1] "Fit elastic net..."


Warning message in train.default(x, y, weights = w, ...):
“The metric "Accuracy" was not in the result set. ROC will be used instead.”

[1] "Get coefficients..."


Warning message in perf_list[i] <- performance_metrics(actual = test.data2$HRDeficient, :
“number of items to replace is not a multiple of replacement length”

In [61]:
res


30 x 1 sparse Matrix of class "dgCMatrix"
                                                           1
(Intercept)                                     1.6367585650
BRCA1_Germline_SNV1                             0.5037176562
BRCA1_Somatic_SNV1                              .           
BRCA1_LOF1                                      .           
BRCA2_Germline_SNV1                             .           
BRCA2_Somatic_SNV1                              .           
BRCA2_LOF1                                      .           
BRCA1_LOH1                                      .           
BRCA2_LOH1                                      .           
BRCA1_INV_SPAN1                                 .           
BRCA2_COPY_GAIN1                                .           
BRCA1status_compoundSNV + deletion (same gene)  .           
BRCA2status_compoundSNV + deletion (same gene)  .           
Double_delDouble deletion                       .           
BRCA1_pro_meth1                            